# Contributing scenes metadata for a region in a Planet Basemap

This example demonstrates how to use the [Planet APIs](https://developers.planet.com/docs/apis/) to retrieve *contributing scene metadata* for an area of interest or region of a Planet basemap. Scene metadata includes geographic footprints as well as other information about the scene like the type of instrument, capture times, azimuth etc.

[Planet Basemaps](https://www.planet.com/products/basemap/) are created by compositing and mosaicking individual scenes, and it's often useful to see which scenes were used to create the basemap. Basemap data is stored as a collection of "quads", geotiff files spanning a regular grid, and each [representation of a quad on the Basemaps API links](https://developers.planet.com/docs/basemaps/reference/#operation/getQuad) to information on which scenes were used in it's creation.

Using the [Planet Basemaps API](https://developers.planet.com/docs/basemaps/) we can search for the quads that comprise a region in a particular basemap, and then use the [Planet Data API](https://developers.planet.com/docs/apis/data/) to retrieve the complete scene metadata that describes each scene present in each quad.


In this example, we use the [Python](https://www.python.org/) programming language with some basic packages like [requests](https://requests.readthedocs.io/en/latest/) to help us interact with the Planet APIs to download the data we want, outputting our result as a [GeoJSON FeatureCollection](https://geojson.org/) whose features are each one of the contributing scenes to the region for the basemap. The scene metadata format can be see in [this response sample](https://developers.planet.com/docs/apis/data/reference/#operation/GetItem) from the Planet Data API. At the end of this notebook, we've combined the steps to create a small utility function that can be used to get contributing scenes metadata for a given basemap and region.

Basemap quads and their contributing scene information can also be viewed in the [Planet Basemaps Viewer](https://www.planet.com/basemaps/) web application, although this script makes it easier to get data for many quads and larger regions.

### Import Pacakges

In [1]:
# Import packages
import os
import json
import requests
from requests.auth import HTTPBasicAuth

### Setup Authentication to Planet APIs

In [2]:
# if your Planet API Key is not set as an environment variable, you can paste it below
if 'PL_API_KEY' in os.environ:
    API_KEY = os.environ['PL_API_KEY']
else:
    API_KEY = 'PASTE_YOUR_API_KEY_HERE'
    os.environ['PL_API_KEY'] = API_KEY

# We can use our API key as the "username" in the basic auth method for our requests
auth = HTTPBasicAuth(API_KEY, '')

### Use the Basemaps API to get quads

In [3]:
# Setup a base url for the Planet Basemaps API
BASE_URL = 'https://api.planet.com/basemaps/v1/mosaics/'

Next we can make a request to get our basemap. We'll need to either know or retrieve the basemap's Id. In the cell below, we can find the basemap's Id by using the `name__is` query parameter with the list [`/mosaics`](https://developers.planet.com/docs/basemaps/reference/#operation/listMosaics) endpoint on the Basemaps API. If you already know the Id of the basemap, you can skip this part and set the `BASEMAP_ID` in subsequent cells.

*Note: "basemaps" and "mosaics" can be referred to interchangeably* 

In [4]:
# Get basemap by name
BASEMAP_NAME = 'global_monthly_2022_05_mosaic'

# NOTE: Alternatively you could use 'name__contains' query parameter to search the basemap by name 
res = requests.get(url=BASE_URL, auth=auth, params={'name__is': BASEMAP_NAME}).json()

# Inspect the response
#print(json.dumps(res, indent=1))

# Get the first result from the response, this should be our basemap
mosaic = res['mosaics'][0]

# Get the basemap ID
BASEMAP_ID = mosaic['id']

print('Basemap Name: {}\nId: {}\nLink: {}'.format(mosaic['name'], BASEMAP_ID, mosaic['_links']['_self']))

Basemap Name: global_monthly_2022_05_mosaic
Id: 045711ab-a3c1-455a-8a2e-b9ff3b699f8a
Link: https://api.planet.com/basemaps/v1/mosaics/045711ab-a3c1-455a-8a2e-b9ff3b699f8a?api_key=PLAKd4412d77080146a8bdf3e629239a7b37


### Define our area of interst

We'll use a [GeoJSON geometry](https://datatracker.ietf.org/doc/html/rfc7946#section-3.1) to define our Area of Interest (AOI) to bound our search for quads in the basemap. The AOI can either be defined manually, or imported from a GeoJSON file on our computer.

In [5]:
# Define aoi using a GeoJSON geometry (Polygon in this case)
AOI = {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -93.12973022460938,
              40.80029619806279
            ],
            [
              -92.977294921875,
              40.80029619806279
            ],
            [
              -92.977294921875,
              40.91247477328731
            ],
            [
              -93.12973022460938,
              40.91247477328731
            ],
            [
              -93.12973022460938,
              40.80029619806279
            ]
          ]
        ]
      }

# Alternatively, we can load a GeoJSON file from our computer
#with open('path/to/my.geojson', 'r') as f:
#    aoi = json.load(f)['geometry']

# Inspect our AOI
#print(json.dumps(AOI, indent=1))

### Get Basemap Quads that intersect our region

Now that we have a region, we can setup a POST request the `/quads/search` endpoint on the Mosaics API and pass our region geometry in order to get back a list of quads that intersect our region. Quads on the Mosaics API follow the format in the response sample section [here](https://developers.planet.com/docs/basemaps/reference/#operation/getQuad).

In [6]:
# Construct the quad search url and pass our aoi as the json body to the POST request
# https://api.planet.com/basemaps/v1/mosaics/BASEMAP_ID/quads/search
quads_url = BASE_URL + BASEMAP_ID + '/quads/search'
quads_res = requests.post(url=quads_url, auth=auth, json=AOI).json()

# The "items" list in the response contains the list on intersected quads
quads = quads_res['items']

# Handle pagination (when there are many quad results... default page_size is 50 items per page)
next_page = quads_res["_links"].get("_next")
while next_page:
    print('Getting Additional Page of Quads:\n {}'.format(next_page))
    paged_res = requests.get(url=next_page, auth=auth).json()
    quads = quads + paged_res['items']
    next_page = paged_res["_links"].get("_next")

print("\nFound {} quads that intersect region.".format(len(quads)))
    
# Inspect response
#print(json.dumps(quads_res, indent=1))


Found 4 quads that intersect region.


### Get the scene metadata for each contributing scene for every quad

Finally, we can create a GeoJSON FeatureCollection that will be used to store all the contributing scene metadata we're retrieving. 

Then we can process each intersecting quad and follow the `items` link from the `_links` section which will give us a list of urls to each contributing scene on the Planet Data API for the quad. This may take some time if our region or number of quads is large, because this means there will likely be a large number of contributing scenes.

In [7]:
# Setup a GeoJSON FeatureCollection to contain all contributing scenes
contributing_scenes = {
  "type": "FeatureCollection",
  "features": []
}

In [8]:
# Get all contributing scenes for each quad

# Loop through our intersecting quads
for quad in quads:
    
    # Get the contributing scenes links for each quad
    contrib_scenes_url = quad['_links']['items']
    contrib_scenes_links = requests.get(url=contrib_scenes_url, auth=auth).json()['items']
    
    # Get each of the contributing scenes' metadata
    for link in contrib_scenes_links:
        scene_metadata = requests.get(url=link['link'], auth=auth).json()
        
        # Inspect each scene
        #print('\nScene Id: {}'.format(scene_metadata['id']))
        #print(scene_metadata)
        
        # Add each scene metadata to the FeatureCollection
        contributing_scenes['features'].append(scene_metadata)

print("Found {} contributing scenes".format(len(contributing_scenes['features'])))
        
# Inspect Contributing Scenes GeoJSON
#print(json.dumps(contributing_scenes, indent=1))

Found 17 contributing scenes


#### Save the list of contributing scene metadata

We can now save our list of contributing scene metadata as a GeoJSON file:

In [9]:
with open('contributing-scenes-metadata.geojson', 'w') as outfile:
    json.dump(contributing_scenes, outfile)

### Conclusion

We've demonstrated how to retrieve contributing scene metadata for a region in a Planet Basemap! 

The GeoJSON file containing the list of scenes includes footprint geometies that can be visualized with most GIS viewers and applications such as [QGIS](https://www.qgis.org/) or [geojson.io](http://geojson.io/), or used in scripts and workflows built in another notebook like this one. 

Furthermore, you can use the `_links` section in each of the contributing scene metadata to activate and download the scenes, or view them using the [Planet Data API](https://developers.planet.com/docs/apis/data/) or [Planet Web Tile Services](https://developers.planet.com/docs/basemaps/tile-services/), provided you have access. 

Note that not all contributing scenes will intersect with your region, since parts of a quad may lay outside of the intersection, although you can further intersect your region with the list of scenes to identify only scenes that intersect your region or area of interest. 



### Sample Utility

Below we've provided a sample utility function to get contributing scenes for a region in a basemap that brings together the steps we followed in this notebook.

The utility takes: 
* A `basemap` parameter, which can be a basemap name or id
* A `region` parameter, which can be a GeoJSON geometry python dict, or a path to a valid GeoJSON Feature file.
* A `save` parameter, indicating whether to save the output to a GeoJSON file.

In [10]:
def get_contributing_scenes_for_basemap_region(basemap, region, save=False):
    '''
    Gets contributing scenes metadata for a defined region in a basemap.
    
    :param string basemap: The basemap name or ID.
    :param string region: The area of interest or region geometry. 
                          Can be a dict containing a GeoJSON geometry or a GeoJSON Feature.
    :param bool save: Whether or not to save an output GeoJSON file.
                      By default, does not save an output file.
    :returns dict: A GeoJSON FeatureCollection dict with contributing scenes
    '''
    
    # Create a session for our API requests
    s = requests.Session()
    s.auth = HTTPBasicAuth(os.getenv('PL_API_KEY'), '')
    
    BASE_URL = 'https://api.planet.com/basemaps/v1/mosaics/'
    
    # Use the mosaic name to retrieve the basemap
    res = s.get(url=BASE_URL, params={'name__is': basemap}).json() 

    if len(res['mosaics']) >= 1:
        # Get the first result from the response, this should be our basemap
        mosaic = res['mosaics'][0]

    else:
        # Use a mosaic id to retrieve the basemap
        mosaic = s.get(url=BASE_URL + basemap).json()
        
        if 'message' in mosaic:
            print(mosaic['message'])
            return
    
    print('Mosaic Name: {}\n  Mosaic Id: {}\n   API Link: {}'.format(mosaic['name'], mosaic['id'], mosaic['_links']['_self']))
   
    # Get the Geometry, if region is a string, open a file
    if isinstance(region, str):
        with open(region, 'r') as f:
            aoi = json.load(f)['geometry']
    elif isinstance(region, dict):
        aoi = region
    else:
        print("Please provide a valid region (JSON dict or path to GeoJSON file)!")
        return
    
    # Construct the quad search url and pass our aoi as the json body to the POST request
    # https://api.planet.com/basemaps/v1/mosaics/BASEMAP_ID/quads/search
    quads_res = s.post(url=BASE_URL + mosaic['id'] + '/quads/search', json=aoi).json()

    # The "items" list in the response contains the list on intersected quads
    quads = quads_res['items']
    
    # Handle pagination (when there are many quad results... default page_size is 50 items per page)
    next_page = quads_res["_links"].get("_next")
    while next_page:
        paged_res = s.get(url=next_page).json()
        quads = quads + paged_res['items']
        next_page = paged_res["_links"].get("_next")

    print("\nFound {} quads that intersect region.".format(len(quads)))
    
    # Setup a GeoJSON FeatureCollection to contain all contributing scenes
    contributing_scenes = {
      "type": "FeatureCollection",
      "features": []
    }
    
    # Get all contributing scenes for each quad
    # Loop through our intersecting quads
    for quad in quads:

        # Get the contributing scenes links for each quad
        contrib_scenes_url = quad['_links']['items']
        contrib_scenes_links = requests.get(url=contrib_scenes_url).json()['items']

        # Get each of the contributing scenes' metadata
        for link in contrib_scenes_links:
            scene_metadata = s.get(url=link['link']).json()
            
            # Add each scene metadata to the FeatureCollection
            contributing_scenes['features'].append(scene_metadata)
    
    print("Found {} contributing scenes".format(len(contributing_scenes['features'])))
    
    if save:
        # Save the contributing scenes list to a file
        filename = mosaic['name'] + '-contrib-scenes.geojson'
        with open(filename, 'w') as outfile:
            json.dump(contributing_scenes, outfile)
        print("Saved contributing scenes to file: {}".format(filename))
    else:
        # Report output
        print('\nContributing Scenes GeoJSON FeatureCollection:')
        
    return
    # Note: un-comment the below line to see the full metadata for all contributing scenes
    # return contributing_scenes

In [11]:
# Example Usage:
# Get contributing scenes for a region in the May 2022 Global Monthly basemap and save the output
get_contributing_scenes_for_basemap_region('global_monthly_2022_05_mosaic', AOI, True)

Mosaic Name: global_monthly_2022_05_mosaic
  Mosaic Id: 045711ab-a3c1-455a-8a2e-b9ff3b699f8a
   API Link: https://api.planet.com/basemaps/v1/mosaics/045711ab-a3c1-455a-8a2e-b9ff3b699f8a?api_key=PLAKd4412d77080146a8bdf3e629239a7b37

Found 4 quads that intersect region.
Found 17 contributing scenes
Saved contributing scenes to file: global_monthly_2022_05_mosaic-contrib-scenes.geojson
